In [7]:
%load_ext autoreload
%autoreload 2

In [7]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen 
from src.sentiment.senti_utils import *
from src.sentiment.process_tweets import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [15]:
city = Path("../data/processed/tweets/DC_Washington_1150000.json")
tweets = load_tweets(city)
park_tweets, control_tweets, park_names = park_control_split(tweets)

Loaded 1090463 tweets
Returned 1089010 unique tweets


In [ ]:
## standard sentiment
p_ngrams, c_ngrams, stops = geotweets_to_ngrams(city)
print("Park Ngrams, Ctl Ngrams", sum(p_ngrams.values()), sum(c_ngrams.values()))
print(city.stem, ": Park: {:05.3f} | Control {:05.3f}".format(senti(p_ngrams, stops), senti(c_ngrams,stops)))

In [ ]:
## labmt sentiment estimate, for shifting 
PVec, CVec,pStops = getVecs(city)
PVec, Pval = labmtSimpleSentiment(PVec,pStops)
CVec, Cval = labmtSimpleSentiment(CVec,pStops)
print(city.stem, ": Park: {:05.3f} | Control {:05.3f}".format(Pval, Cval))
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
#shiftHtml(labMTvector,labMTwordList, CVec, PVec, "shifts/" + str(city.stem)+".html")

In [35]:
city = Path("../data/processed/tweets/FL_Jacksonville_1235000.json")
tweets = load_tweets(city)
park_tweets, control_tweets, park_names = park_control_split(tweets)
stops  = get_park_stopwords(park_names)

In [36]:
print(len(park_tweets))

7234


In [37]:
# Now we bootstrap a vector of 100 seniments, by choosing 80% of the tweets in each group
# Test bootstrap 
for n in np.arange(.1,1,.1):
    bumps = bootstrap_sentiment(park_tweets, control_tweets, stops,sample=n,runs=10)
    print("{:.0f}% sampled: Mean Park: {:.4f} with range {:.4f}".format(n*100, np.mean(bumps), max(bumps)-min(bumps)))

#print(min(park_sentis), np.mean(park_sentis), max(park_sentis))
#print(min(control_sentis), np.mean(control_sentis), max(control_sentis))

10% sampled: Mean Park: 0.0935 with range 0.1228
20% sampled: Mean Park: 0.0752 with range 0.1350
30% sampled: Mean Park: 0.0903 with range 0.0745
40% sampled: Mean Park: 0.0847 with range 0.0674
50% sampled: Mean Park: 0.0866 with range 0.0729
60% sampled: Mean Park: 0.0788 with range 0.0554
70% sampled: Mean Park: 0.0786 with range 0.0369
80% sampled: Mean Park: 0.0807 with range 0.0249
90% sampled: Mean Park: 0.0805 with range 0.0189


In [ ]:
# Now we bootstrap a vector of 100 seniments, by choosing 80% of ngrams in each group
tweets = load_tweets(city)
park_tweets, control_tweets, park_names = park_control_split(tweets)
stops  = get_park_stopwords(park_names)
park_sentis, control_sentis = bootstrap_sentiment_ngrams(park_tweets, control_tweets, stops,10)

print(min(park_sentis), np.mean(park_sentis), max(park_sentis))
print(min(control_sentis), np.mean(control_sentis), max(control_sentis))

Loops through all processed tweets and bootstrap results

In [ ]:
tweet_dir = Path("../data/processed/tweets/")
results = {}
for city in tweet_dir.iterdir():
    tweets = load_tweets(city)
    park_tweets, control_tweets, park_names = park_control_split(tweets)
    stops  = get_park_stopwords(park_names)
    park_sentis, control_sentis = bootstrap_sentiment(park_tweets, control_tweets, stops,20)
    results[city.stem[:-8]] = {"park_sentiment":park_sentis, "control_sentiment":control_sentis}
    


In [ ]:
# save results dictionary to file
with open("results/city_bootstrap.json", 'w') as f:
    json.dump(results, f)    